In [1]:
#Import Necessary Modules
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json

In [2]:
source = requests.get('https://www.btk.gov.tr/yillik-il-istatistikleri')

In [3]:
soup = BeautifulSoup(source.text, 'lxml')

In [4]:
files = soup.find_all(name='div', attrs={'class':'SmTqk'})[0]

In [5]:
links = {}
for a in files.find_all(name='a'):
    link = a['href']
    if link.endswith('.xlsx'):
        links[a.text] = link

In [6]:
links

{'2016 - 2021': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/il-bazinda-yillik-istatistik-bulteni-v1.xlsx',
 '2015 - 2020': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/elektronik-haberlesme-sektorune-iliskin-il-bazinda-yillik-istatistik-bulteni.xlsx',
 '2014 - 2019': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/il-bazinda-yillik-istatistik-bulteni-20-05-2020.xlsx',
 '2013 - 2018': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/il-bazinda-yillik-istatistik-bulteni1.xlsx',
 '2012 - 2017': 'https://www.btk.gov.tr/uploads/pages/yillik-il-istatistikleri/il-bazinda-yillik-istatistik-bulteni.xlsx',
 '2011 - 2016': 'https://www.btk.gov.tr/uploads/pages/slug/ehsyib-2011-2016-5a3a48147c468.xlsx',
 '2010 - 2015': 'https://www.btk.gov.tr/uploads/pages/slug/ehsyib-2010-2015.xlsx',
 '2009 - 2014': 'https://www.btk.gov.tr/uploads/pages/slug/ehsyib-2009-2014.xlsx',
 '2008 - 2013': 'https://www.btk.gov.tr/uploads/pages/slug/ehsyib-2

In [7]:
def get_dataframe(link):
    data = pd.read_excel(link, sheet_name='İl Bazlı Veri')
    data = data.iloc[2:,:]
    data['detail'] = data.iloc[:,1].fillna('') + data.iloc[:,2].fillna('')
    data['city'] = data.iloc[:,0]
    data = data.iloc[:,3:]
    data['city'] = data['city'].fillna(method='ffill')
    data.dropna(subset=[x for x in data.columns if str(x) not in ['city','detail']], axis=0, inplace=True)
    data['detail'] = data['detail'].str.strip(r'[()]')
    return data[data.columns[::-1]].reset_index(drop=True)[:-1]

In [37]:
df = pd.DataFrame()
for key, link in links.items():
    print(key)
    data = get_dataframe(link)
    if df.shape[0] == 0:
        df = data
    else:
        cols_to_use = data.columns.difference(df.columns[2:])
        df = df.merge(data[cols_to_use], how='outer', on=['city', 'detail'])

2016 - 2021
2015 - 2020
2014 - 2019
2013 - 2018
2012 - 2017
2011 - 2016
2010 - 2015
2009 - 2014
2008 - 2013
2007 - 2012


In [45]:
df.sort_values(by='city', inplace=True)
df.reset_index(drop=True, inplace=True)

In [58]:
df = df.melt(id_vars=['city','detail'], var_name=['year'], value_name='value')

In [82]:
df = df.pivot(index=['city','year'], columns='detail').reset_index()

In [88]:
df.columns = df.columns[:2].get_level_values(0).tolist() + df.columns[2:].get_level_values(1).tolist()

In [91]:
df.columns = [col.lower() for col in df.columns]

In [92]:
df.head()

,city,year,cable,fiber,mobile broadband dedicated,number of broadband subscriptions - total,number of cable tv subscriptions,number of fixed broadband subscriptions - total,number of fixed telephony access lines,number of mobile broadband subscriptions - total,...,number of mobile telephony subscriptions - 2g,number of mobile telephony subscriptions - 3g,number of mobile telephony subscriptions - total,number of payhones,other,population,santral capacity for fixed telephony,standard mobile broadband,the length of fiber-km,xdsl
0,ADANA,2007,NaN,NaN,NaN,NaN,41115,NaN,402348,NaN,...,NaN,NaN,1759062,2022,NaN,2006650,523425,NaN,NaN,NaN
1,ADANA,2008,NaN,NaN,NaN,NaN,40400,NaN,369043,NaN,...,NaN,NaN,1889383,1986,NaN,2026319,534366,NaN,NaN,NaN
2,ADANA,2009,NaN,NaN,NaN,NaN,43826,NaN,332356,NaN,...,NaN,NaN,1768410,1912,NaN,2062226,525061,NaN,NaN,NaN
3,ADANA,2010,NaN,NaN,NaN,NaN,51835,NaN,310454,NaN,...,1172924,542370,1715294,1877,NaN,2085225,529486,NaN,NaN,NaN
4,ADANA,2011,30406,21286,29052,561382.40352,57089,167692.40352,270622,393690,...,983747,NaN,1830386,1786,126.40352,2108805,596315,364638,NaN,115874


In [93]:
df.to_csv('final_merged.csv', encoding='utf-8-sig')